# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import cvxopt
import numpy as np

iris = load_iris()
data_set = iris.data
labels = iris.target  
print(np.asarray(data_set).shape)

(150, 4)


In [32]:
def build_kernel(data_set, kernel_type='linear', degree=2):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.linalg.matrix_power(kernel, degree)
    return kernel

In [33]:
build_kernel(train_data_set, kernel_type='poly', degree=2)

array([[229382.4603, 169340.9093, 194767.5298, ..., 201990.1791,
        171105.3501, 250841.2972],
       [169340.9093, 125642.4972, 144493.9497, ..., 149901.869 ,
        126973.778 , 184985.3786],
       [194767.5298, 144493.9497, 166174.3575, ..., 172392.2437,
        146024.6959, 212765.6728],
       ...,
       [201990.1791, 149901.869 , 172392.2437, ..., 178847.5088,
        151491.2048, 220639.497 ],
       [171105.3501, 126973.778 , 146024.6959, ..., 151491.2048,
        128320.1897, 186906.204 ],
       [250841.2972, 184985.3786, 212765.6728, ..., 220639.497 ,
        186906.204 , 274370.3014]])

In [34]:
build_kernel(train_data_set, kernel_type='poly', degree=1)

array([[58.83, 41.31, 47.53, ..., 49.19, 41.63, 64.95],
       [41.31, 33.06, 37.97, ..., 39.56, 33.5 , 44.38],
       [47.53, 37.97, 43.65, ..., 45.39, 38.52, 51.15],
       ...,
       [49.19, 39.56, 45.39, ..., 47.42, 40.04, 52.7 ],
       [41.63, 33.5 , 38.52, ..., 40.04, 34.01, 44.75],
       [64.95, 44.38, 51.15, ..., 52.7 , 44.75, 72.2 ]])

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [35]:
def choose_set_for_label(data_set, labels):   
    N = len(np.unique(labels))
    combinations = []
    for i in range(N):
        for j in range(N):
            if i!=j and i<=j:
                x = np.concatenate(
                    [
                        data_set[labels==i],
                        data_set[labels==j]
                    ], axis = 0
                )
                y = np.concatenate(
                    [
                        np.ones(len(data_set[labels==i])),
                        np.ones(len(data_set[labels==j])) * (-1)
                    ], axis = 0
                )
                
                combinations.append(train_test_split(x, y, test_size=0.2, random_state=15))
    return combinations

combinations = choose_set_for_label(data_set, labels)

In [36]:
def get_labels_count(labels):
    return len(np.unique(labels))

In [37]:
# Use the code that we have implemented earlier:

In [38]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [42]:
# modify this part 
from sklearn.metrics import accuracy_score
N = get_labels_count(labels)
COMBINATIONS_NUM = N*(N-1)//2
for i in range(COMBINATIONS_NUM):
    X_train, X_test, y_train, y_test = combinations[i]
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(
        X_train, y_train, kernel_type='rbf')
    predicted = classify_rbf(X_test, X_train, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted = list(predicted.astype(int))
    print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  9.6305e+01 -1.2289e+03  2e+03  2e-01  2e-15
 1:  5.9143e+01 -1.2031e+02  2e+02  5e-03  2e-15
 2:  7.0898e+00 -1.6497e+01  2e+01  2e-16  2e-15
 3: -5.2057e-01 -3.7668e+00  3e+00  2e-16  1e-15
 4: -1.1712e+00 -1.8374e+00  7e-01  2e-16  4e-16
 5: -1.3952e+00 -1.6846e+00  3e-01  2e-16  2e-16
 6: -1.4671e+00 -1.5679e+00  1e-01  2e-16  2e-16
 7: -1.5060e+00 -1.5164e+00  1e-02  2e-16  2e-16
 8: -1.5105e+00 -1.5106e+00  1e-04  2e-16  2e-16
 9: -1.5105e+00 -1.5105e+00  1e-06  2e-16  2e-16
Optimal solution found.
0.15
     pcost       dcost       gap    pres   dres
 0:  1.0960e+02 -1.2787e+03  2e+03  1e-01  2e-15
 1:  6.2790e+01 -1.3211e+02  2e+02  6e-03  2e-15
 2:  7.3654e+00 -1.8102e+01  3e+01  2e-16  3e-15
 3: -7.5515e-01 -4.2288e+00  3e+00  2e-16  9e-16
 4: -1.4405e+00 -2.3676e+00  9e-01  1e-16  4e-16
 5: -1.7465e+00 -2.0630e+00  3e-01  2e-16  2e-16
 6: -1.8105e+00 -1.9114e+00  1e-01  2e-16  2e-16
 7: -1.8438e+00 -1.8684e+00  2e-02  2e-16 